# Chapter 12 - Introducing deep learning for time series forecasting

Prerequisite concepts:
- Activation functions
- Loss functions
- Batches
- Layers
- Epochs

## 12.1 When to use deep learning for time series forecasting

## 12.2 Exploring the different types of deep learning models